## Text-to-Image Generation using Stable Diffusion
- huggingface, pytorch 사용
- 참조
  * https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/overview
  * https://huggingface.co/docs/diffusers/ko/using-diffusers/write_own_pipeline 

In [ ]:
import torch
import time
from diffusers import StableDiffusionPipeline
from IPython.display import Image

In [ ]:
# 학습된 stable diffusion v1.5 모델 경로입니다.
SD_PATH = "/group-volume/sr_edu/AI-Application-Specialist-Vision-Dataset/hf-models/stable-diffusion-v1-5"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# diffusers 에서 제공하는 pipeline을 이용하여 text to image pipeline을 생성합니다
# 시간이 걸립니다.
pipe = StableDiffusionPipeline.from_pretrained(SD_PATH)
pipe = pipe.to(device)
print(f"Stable Diffusion v1.5 has been loaded on {device} device")

seed = 12345
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

# 모델 구조
- Stable Diffusion 은 크게 CLIP, UNet, VAE라는 세 가지 인공신경망으로 구성되어 있습니다.
  - CLIP: 텍스트 입력을 토큰으로 변환
  - UNet: 토큰을 기반으로 무작위로 생성된 노이즈를 디노이징하여 이미지 Latent 를 생성
  - VAE: Pixel Space 를 Latent Space 를 만드는 인코더와 이것을 Pixel Space 로 되돌리는 디코더

In [ ]:
Image('./images/stable_diffusion.jpg')

In [ ]:
!ls {SD_PATH}

In [ ]:
print(pipe)

# 이미지를 생성해보자

In [ ]:
# Text to Image 에 사용할 프롬프트를 pipeline에 넣어주고 이미지 결과를 받습니다.
prompt = "a photo of an astronaut riding a horse on mars"
pipe(prompt).images[0]

In [ ]:
# Text to Image 에 사용할 프롬프트를 pipeline에 넣어주고 이미지 결과를 받습니다.
prompt = "a photo of an astronaut riding a horse on mars by Van Gogh"
pipe(prompt).images[0]

# 프롬프트 엔지니어링
- 프롬프트를 잘 만드는 것은 원하는 이미지를 얻기 위해 가장 기본이 되는 step 입니다.
- 프롬프트를 잘 만들기 위해서…
  - 기존 프롬프트를 재활용 해보기 
    - [civitai.com](https://civitai.com/images), [openart.ai](https://openart.ai/discovery) 와 같은 생성형 이미지 공유사이트를 참고하기
  - chatGPT 를 사용해서 프롬프트 만들어보기
  - openart.ai에서 만든 [프롬프트 북](https://cdn.openart.ai/assets/Stable%20Diffusion%20Prompt%20Book%20From%20OpenArt%2011-13.pdf) 읽어보기

In [ ]:
# 원하는 텍스트 프롬프트를 작성하고 이미지를 생성해보세요.
prompt = 
pipe(prompt).images[0]

# Stable diffusion 파라미터들

- 위에처럼 간단하게 pipe(prompt)로 이미지를 생성할 수 있지만, 기본 파라미터 외에도 조정할 수 있는 다양한 파라미터가 있습니다.
- 종류 및 설명
  - prompt: T2I에 사용할 텍스트 프롬프트
  - height, width: 생성할 이미지 사이즈
  - num_inference_steps: 몇번의 inference step을 진행 할 것인가. 낮을수록 낮은 퀄리티
  - guidance scale: text prompt로 guide를 주는 정도. 낮을수록 텍스트와 거리가 있는 이미지가 생성 될 수 있음. 높을수록 텍스트 반영도가 높음
  - num_images_per_prompt: 한 번에 몇 장 생성할지
  - generator: seed를 주어 deterministic하게 생성할 수 있음


In [ ]:
# 기본 파라미터 셋팅
st = time.time()
image = pipe(
    prompt=prompt,
    height=512,
    width=512,
    num_inference_steps=50,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    generator=torch.Generator(device).manual_seed(seed),
).images[0]
print(time.time()-st)
image

In [ ]:
# num_inference_step을 줄여보자
st = time.time()
image = pipe(
    prompt=prompt,
    height=512,
    width=512,
    num_inference_steps=10,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    generator=torch.Generator(device).manual_seed(seed),
).images[0]
print(time.time()-st)
image

In [ ]:
# num_inference_step을 줄여보자
st = time.time()
image = pipe(
    prompt=prompt,
    height=512, 
    width=512, 
    num_inference_steps=30,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    generator=torch.Generator(device).manual_seed(seed),
).images[0]
print(time.time()-st)
image

- 위 예시처럼, 경우에 맞게 파라미터를 조정 할 수 있습니다.
  - 빠르게 샘플링해서 경향성을 봐야하는 경우 30 steps 으로도 충분할 수 있습니다.
  - 좋은 퀄리티의 이미지를 얻기 위해서는 default 값인 50 steps를 권장합니다.

# NSFW (Not safe for work) 기능

In [ ]:
prompt = "naked"
pipe(prompt).images[0]

- 하지만 완벽하진 않아서, NSFW 가 아닐 것임에도 NSFW로 검출되는 경우가 있습니다.

In [ ]:
pipe(
    prompt="Mark Zuckerberg",
    height=512,
    width=512,
    num_inference_steps=30,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    generator=torch.Generator(device).manual_seed(seed),
).images[0]

## NSFW 우회하는 방법
1. 일시적으로 우회가 필요한 것이라면, pil image로 결과를 받는 것이 아닌 latent 형태로 결과를 받는다.
2. 아예 NSFW 가 필요 없다면, 처음 모델을 로드할 때 requires_safety_check를 해제한다.

In [ ]:
# 1. 일시적으로 우회가 필요한 것이라면, pil image로 결과를 받는 것이 아닌 latent 형태로 결과를 받는다.

latents = pipe(
    prompt="Mark Zuckerberg",
    height=512,
    width=512,
    num_inference_steps=30,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    generator=torch.Generator(device).manual_seed(seed),
    output_type="latent",
).images

image = pipe.vae.decode(
    latents / pipe.vae.config.scaling_factor, return_dict=False
)[0]
do_denormalize = [True] * image.shape[0]
image = pipe.image_processor.postprocess(
    image.detach(), output_type="pil", do_denormalize=do_denormalize
)[0]
image

In [ ]:
print(pipe)

In [ ]:
# 2. 아예 NSFW 가 필요 없다면, 처음 모델을 로드할 때 requires_safety_check를 해제한다.

pipe = StableDiffusionPipeline.from_pretrained(SD_PATH, requires_safety_checker=False, safety_checker=None, low_cpu_mem_usage=False)
pipe = pipe.to(device)

print(f"Stable Diffusion v1.5 has been loaded on {device} device")

In [ ]:
print(pipe)

In [ ]:
pipe(
    prompt="Mark Zuckerberg",
    height=512,
    width=512,
    num_inference_steps=30,
    guidance_scale=7.5,
    num_images_per_prompt=1,
    generator=torch.Generator(device).manual_seed(seed),
).images[0]